In [1]:
%load_ext autoreload
%autoreload 2

With this notebook, I intend to clean DB5 and discern which features are of interest for the follow-up of the analyses. Once the optimization is done, and potential algorithms for clustering and classification are developed, the next step would be to subject these features to examination.

In [2]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import statsmodels.api as sm
import matplotlib.patches as mpatches

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, precision_recall_fscore_support

pd.set_option('display.max_columns', None)
path = "data/"

In [3]:
# Obtained from Optimization
min_subset_ids = pd.read_csv(path+"id_vs_frequency_decreasing_ds.csv")

raw_DB5 = pd.read_excel(path+"HDB5V2.3.xlsx")


DB2 = pd.read_csv(path+"DB2P8.csv")
DB5 = pd.read_csv(path+"SELDB5_SVD.csv", low_memory=False) 
DB5 = DB5[DB5["PHASE"].isin(['HGELM', 'HSELM', 'HGELMH', 'HSELMH'])]

# There are shots missing in DB5 from DB2P8
missing_shots = DB2[~DB2.id.isin( DB5.id.values )].reset_index(drop=True)
DB5 = pd.concat([DB5, missing_shots], axis=0, ignore_index=True)

# Labeling shots that had great impact in decreasing alpha_R
DB5.insert(loc=2,column="label",value=[0]*len(DB5))
DB5.loc[(DB5[DB5.id.isin(min_subset_ids.id)].index), "label"] = 1

print(
    f"{ round( (len(min_subset_ids)/len(DB5))*100     ,2)  }% of the data decreased alpha_R\n" + 
    f"{ round( (1 - len(min_subset_ids)/len(DB5))*100 ,2)  }% of the data did not decrease alpha_R"
)

40.69% of the data decreased alpha_R
59.31% of the data did not decrease alpha_R


In [4]:
raw_DB5.shape

(14153, 192)

In [5]:
raw_DB5.DATE.astype(str).apply(lambda x: x[:4]).unique()

array(['1982', '1983', '1984', '1985', '1986', '1990', '1994', '1995',
       '1996', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2008', '2009', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017', '1997', '1987', '1989', '1991', '1992',
       '1993', '1988'], dtype=object)

In [6]:
sorted(DB2.DATE.astype(str).apply(lambda x: x[:4]).unique())

['1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997']

In [7]:
sorted(DB5.DATE.astype(str).apply(lambda x: x[:4]).unique())

['1982',
 '1983',
 '1984',
 '1985',
 '1986',
 '1987',
 '1988',
 '1989',
 '1990',
 '1991',
 '1992',
 '1993',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2008',
 '2009',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017']

In [8]:
sorted(DB5[~DB5.id.isin(DB2.id.values)].DATE.astype(str).apply(lambda x: x[:4]).unique())

['1989',
 '1990',
 '1994',
 '1995',
 '1996',
 '1997',
 '1998',
 '1999',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2008',
 '2009',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017']

In [9]:
num_features = raw_DB5.select_dtypes(include=['int', 'float']).columns.tolist()
cat_features = raw_DB5.select_dtypes(include=['object']).columns.tolist()

In [10]:
len(num_features)

164

In [11]:
len(cat_features)

28

In [12]:
len(num_features) + len(cat_features)

192

In [14]:
DB5["TAV"].describe()

count    6252.000000
mean     1584.812505
std       781.201366
min       154.322552
25%      1015.637989
50%      1395.060888
75%      2042.758782
max      8484.381925
Name: TAV, dtype: float64

In [21]:
DB5[DB5["TAV"]>8484][["TIV","TEV","TIV","TAUTH"]]

TIV     TEV     TIV  TAUTH
3247  3129.0  3408.0  3129.0  0.508

In [16]:
raw_DB5["TIV"].describe()

count    4.369000e+03
mean     2.353812e+03
std      2.006984e+03
min      2.681000e-07
25%      1.198000e+03
50%      1.911000e+03
75%      2.842000e+03
max      2.086000e+04
Name: TIV, dtype: float64

In [20]:
raw_DB5[raw_DB5["TIV"]>2e+04][["TIV","TEV","TAUTH"]]

TIV     TEV   TAUTH
6223  20860.0  4749.0  0.7791
6379  20300.0  4365.0  0.6214

In [23]:
DB5["TAUTH"].describe()

count    6252.000000
mean        0.177301
std         0.147450
min         0.002236
25%         0.069788
50%         0.125950
75%         0.244868
max         1.321000
Name: TAUTH, dtype: float64

In [26]:
DB5[DB5["TAUTH"]>1][["TIV","TEV","TIV","TAUTH","TOK","DATE"]]

TIV     TEV     TIV  TAUTH  TOK      DATE
3020  2963.0  3193.0  2963.0  1.011  JET  19880506
3021  2726.0  3142.0  2726.0  1.005  JET  19880506
3029     NaN  2617.0     NaN  1.008  JET  19880715
3120  4293.0  2910.0  4293.0  1.195  JET  19901009
3121  5650.0  3470.0  5650.0  1.321  JET  19901009
3123  4711.0  2923.0  4711.0  1.198  JET  19901009
3124  2784.0  2814.0  2784.0  1.208  JET  19901009
3290  2535.0  2944.0  2535.0  1.070  JET  19960920

# INFORMATION IN THE DATABASE

[Description of Features](https://docs.google.com/document/d/19iVcGGwQGzrTmRmv6QWqR_BRhZhJrsfM/edit?usp=sharing&ouid=108518602132499005107&rtpof=true&sd=true)

IMPORTANT

* For all registers, there is no internal transport barrier registered.
* Not all columns might have the same units even if the represent the same; z.B.: Power. 


**IDEA** Make a note on the TOKAMAKS that use a Limiter and the ones that don't. I think not all Tokamaks have a Divertor as well. Check that. :P 

* Apparently, `TFTR` doesn't have a Divertor; but, it is hard to tell. 

#### NOTE ON HEATING AND DIVERTORS

Obtained from: **ADX: a high field, high power density, advanced divertor and RF tokamak** 

[[B. LaBombard, E. Marmar, J. Irby, T. Rognlien, M.Umansky, 2017]](https://www.osti.gov/pages/biblio/1463827)

<img src="data/Images/ADX: a high field-high power_ICRH.png" class="center" width=600>

In [13]:
# DWMHD = DWDIA
plasma_characteristics = ["QCYL5","BEIMHD","PREMAG","LHTIME","HYBRID",
                          "CONFIG","DWDIA","WMHD","TORQ"
                         ] 
TOK_characteristics = ["TOK","DIVNAME","WALMAT","DIVMAT","LIMMAT"]
ELM = ["ELMTYPE","ELMFREQ"]
heating = ["PECRH", "PICRH", "ICSCHEME","AUXHEAT","ENBI"]
impurities = ["EVAP","ZEFF","ZEFFNEO","PRAD",]
 # corrections on power loss | NBI Power lost by unconfined orbits
power = ["PLTH","PFLOSS","POHM"]
temperatures = ["TAV","TEV","TIV"]
# e-density in SOL | total due to NBI| total due to ICRH
fast_particles = ["NESOL","WFFORM","WFICFORM"] 

interesting_features = plasma_characteristics + TOK_characteristics + ELM + heating + \
                       impurities + power + temperatures + fast_particles

In [14]:
physical_variables = ["RHOSTAR","BETASTAR","NUSTAR"]
engineering_variables = ["KAREA","EPS","NEL","IP","MEFF","BT","PLTH","RGEO"]

entropy_features = ['RHOSTAR', 'BETASTAR', 'KAREA', 'EPS', 'NEL', 'TAV', 'QCYL5', 'NUSTAR']
research_features = ['TAUTH','NEL','TAV','BT','RHOSTAR','NUSTAR','BETASTAR']

In [15]:
categorical = ["PREMAG","HYBRID","CONFIG","ELMTYPE",
               "ICSCHEME","AUXHEAT","EVAP"] + TOK_characteristics 

In [16]:
# Removing: HYBRID, LHTIME DIVNAME
plasma_characteristics = ["BEIMHD", "BETASTAR", "CONFIG", "DWDIA", "EPS", "IP", "KAREA",
                          "LCOULOMB", "MEFF", "NUSTAR", "PREMAG", "Q95", "RHOSTAR",
                          "TORQ", "VOL", "WMHD"]
TOK_characteristics = ["AMIN", "BT", "DIVMAT", "LIMMAT", "TOK", "WALMAT"]
ELM = ["ELMFREQ", "ELMTYPE"]
heating = ["AUXHEAT", "ECHMODE", "ICSCHEME", "PECRH", "PELLET", "PICRH"]
impurities = ["ENBI", "EVAP", "PNBI", "POHM", "PRAD", "ZEFF", "ZEFFNEO"]
power = ["PFLOSS", "PLTH"]
temperatures = ["TAV", "TEV", "TIV"]
fast_particles = ["NEL", "NESOL", "OMEGACYCL", "WFFORM", "WFICFORM"]

features = (plasma_characteristics + TOK_characteristics + ELM + heating + 
                       impurities + power + temperatures  + fast_particles)

len(features)

47

In [17]:
sorted(impurities)

['ENBI', 'EVAP', 'PNBI', 'POHM', 'PRAD', 'ZEFF', 'ZEFFNEO']

In [18]:
"Acknowledgements".upper()

'ACKNOWLEDGEMENTS'

### TREATMENT TO CATEGORICAL DATA

#### Replacement of NaN to UNKNOWN

In [19]:
DB5[categorical] = DB5[categorical].fillna('UNKNOWN')
DB5["ICSCHEME"]   = DB5["ICSCHEME"].str.replace("OFF","NONE",regex=False)

In [20]:
DB5.TOK.unique()

array(['ASDEX', 'AUG', 'AUGW', 'CMOD', 'COMPASS', 'D3D', 'JET', 'JETILW',
       'JFT2M', 'JT60U', 'MAST', 'NSTX', 'PBXM', 'PDX', 'START', 'TCV',
       'TDEV', 'TFTR'], dtype=object)

In [21]:
DB5["DIVNAME"].unique()

array(['DV-IPRE', 'DV-IPOST', 'DV-II-C', 'DIV-I', 'DIV-II', 'DIV-IIb',
       'DIV-IIc', 'DIV-IId', 'DIV-III', 'NONAME', 'DIV1', 'OPEN', 'ADP',
       'RDP', 'MARK0', 'MARKI', 'MARKIIA', 'MARKIIAP', 'MARKGB',
       'MARKGBSR', 'ILW', 'RIB', 'PLATES1', 'UNKNOWN'], dtype=object)

In [22]:
DB5["DIVNAME"] = DB5["DIVNAME"].str.replace("(DIV-I)|(DV-IPRE)|(DV-IPOST)",
                                            "DV-I",regex=True)
DB5["DIVNAME"] = DB5["DIVNAME"].str.replace("(DIV-II)|(DV-IIc)|(DV-II-C)|(DV-IIb)|(DV-IIc)|(DV-IId)|(DV-IId)",
                                            "DV-II",regex=True)
DB5["DIVNAME"] = DB5["DIVNAME"].str.replace("(MARK0)|(MARKI)|(MARKIIA)|(MARKGB)|(MARKGBSR)|"+
                                            "(MARKIA)|(MARKIAP)|(MARKSR)|(MARKA)|(MARKP)",
                                            "MARK",regex=True)
DB5["DIVNAME"].unique()

array(['DV-I', 'DV-II', 'DV-III', 'NONAME', 'DIV1', 'OPEN', 'ADP', 'RDP',
       'MARK', 'MARKIA', 'MARKIAP', 'MARKSR', 'ILW', 'RIB', 'PLATES1',
       'UNKNOWN'], dtype=object)

In [23]:
DB5["DIVNAME"].unique()

array(['DV-I', 'DV-II', 'DV-III', 'NONAME', 'DIV1', 'OPEN', 'ADP', 'RDP',
       'MARK', 'MARKIA', 'MARKIAP', 'MARKSR', 'ILW', 'RIB', 'PLATES1',
       'UNKNOWN'], dtype=object)

In [24]:
power = ["PLTH","PFLOSS"]
temperatures = ["TAV","TEV","TIV"]
fast_particles = ["NESOL","WFFORM","WFICFORM","OMEGACYCL","NEL"] 

In [25]:
sorted(fast_particles)

['NEL', 'NESOL', 'OMEGACYCL', 'WFFORM', 'WFICFORM']

In [26]:
len(fast_particles)

5

In [27]:
strrr = "'WFICFORM', 'WFFORM', 'ZEFFNEO', 'DWDIA', 'RHOSTAR', 'POHM', 'BETASTAR', 'NEL', 'WMHD', 'TAV', 'NUSTAR', 'PFLOSS', 'EPS', 'PLTH', 'QCYL5'"

In [28]:
strrr.replace("'","")

'WFICFORM, WFFORM, ZEFFNEO, DWDIA, RHOSTAR, POHM, BETASTAR, NEL, WMHD, TAV, NUSTAR, PFLOSS, EPS, PLTH, QCYL5'

In [29]:
"INVESTIGATING THE DEPENDENCE ON MACHINE SIZE OF THE ENERGY CONFINEMENT IN TOKAMAKS USING DATA-DRIVEN METHODS".title()

'Investigating The Dependence On Machine Size Of The Energy Confinement In Tokamaks Using Data-Driven Methods'

In [30]:
len(DB5[features].select_dtypes(include=['int', 'float']).columns.tolist())

35

In [31]:
len(DB5[features].select_dtypes(include=['object']).columns.tolist())

12

In [32]:
DB5

ind      id  label    TOK  TOK_ID  DIVNAME    LCUPDATE      DATE  \
0     12405  HDULEH      0  ASDEX     1.0     DV-I  19940125.0  19820622   
1     12406  NAC6N1      0  ASDEX     1.0     DV-I  19940125.0  19820622   
2     12411  U2T1C7      0  ASDEX     1.0     DV-I  19940125.0  19820622   
3     12412  422XQB      0  ASDEX     1.0     DV-I  19940125.0  19820622   
4     12413  WZ9FED      0  ASDEX     1.0     DV-I  19940125.0  19820622   
...     ...     ...    ...    ...     ...      ...         ...       ...   
6247  24964  K19A5H      0   TDEV    14.0   NONAME  19990916.0  19980408   
6248  26411  D26CA1      1   TFTR    13.0   NONAME  19980115.0  19900124   
6249  26412  UPV67I      1   TFTR    13.0   NONAME  19980115.0  19900124   
6250  14027  9GJ6AF      0    AUG     NaN  UNKNOWN         NaN  19960521   
6251  14079  BGDX92      0    AUG     NaN  UNKNOWN         NaN  19960611   

       SHOT   TIME  TIME_ID     T1     T2  AUXHEAT  PHASE   HYBRID      ITB  \
0      5980  1.205   1205.0  1.203  1.207       NB  HGELM  UNKNOWN  UNKNOWN   
1      5980  1.224   1224.0  1.222  1.226       NB  HGELM  UNKNOWN  UNKNOWN   
2      5982  1.188   1188.0  1.186  1.190       NB  HSELM  UNKNOWN  UNKNOWN   
3      5982  1.216   1216.0  1.214  1.218       NB  HGELM  UNKNOWN  UNKNOWN   
4      5982  1.244   1244.0  1.242  1.246       NB  HGELM  UNKNOWN  UNKNOWN   
...     ...    ...      ...    ...    ...      ...    ...      ...      ...   
6247  33171  0.800    800.0    NaN    NaN       EC  HSELM  UNKNOWN  UNKNOWN   
6248  45980  4.600   4600.0    NaN    NaN       NB  HSELM  UNKNOWN  UNKNOWN   
6249  45984  4.200   4200.0    NaN    NaN       NB  HSELM  UNKNOWN  UNKNOWN   
6250   8175  1.502      NaN    NaN    NaN  UNKNOWN  HGELM  UNKNOWN      NaN   
6251   8255  1.651      NaN    NaN    NaN  UNKNOWN  HGELM  UNKNOWN      NaN   

     ITBTYPE  ELMTYPE  ELMFREQ        ELMMAX    ELMDUR        ELMINT  OLTIME  \
0        NaN  UNKNOWN      NaN           NaN       NaN           NaN   1.110   
1        NaN  UNKNOWN      NaN           NaN       NaN           NaN   1.110   
2        NaN  UNKNOWN      NaN           NaN       NaN           NaN   1.115   
3        NaN  UNKNOWN      NaN           NaN       NaN           NaN   1.115   
4        NaN  UNKNOWN      NaN           NaN       NaN           NaN   1.115   
...      ...      ...      ...           ...       ...           ...     ...   
6247     NaN  UNKNOWN      NaN  4.180000e+19  0.000285  1.530000e+16     NaN   
6248     NaN  UNKNOWN      NaN           NaN       NaN           NaN     NaN   
6249     NaN  UNKNOWN      NaN           NaN       NaN           NaN     NaN   
6250     NaN  UNKNOWN      NaN           NaN       NaN           NaN     NaN   
6251     NaN  UNKNOWN      NaN           NaN       NaN           NaN     NaN   

      LHTIME  TPI  ISEQ  MEFF  PGASA  PGASZ  BGASA  BGASZ  BGASA2  BGASZ2  \
0      1.166  6.0  NONE   1.5    2.0    1.0    1.0    1.0     0.0     0.0   
1      1.166  7.0  NONE   1.5    2.0    1.0    1.0    1.0     0.0     0.0   
2      1.165  5.0  NONE   1.5    2.0    1.0    1.0    1.0     0.0     0.0   
3      1.165  6.0  NONE   1.5    2.0    1.0    1.0    1.0     0.0     0.0   
4      1.165  7.0  NONE   1.5    2.0    1.0    1.0    1.0     0.0     0.0   
...      ...  ...   ...   ...    ...    ...    ...    ...     ...     ...   
6247     NaN  0.0  NONE   2.0    2.0    1.0    0.0    0.0     0.0     0.0   
6248     NaN  0.0  NONE   2.0    2.0    1.0    2.0    1.0     0.0     0.0   
6249     NaN  0.0  NONE   2.0    2.0    1.0    2.0    1.0     0.0     0.0   
6250     NaN  NaN   NaN   2.0    NaN    NaN    NaN    NaN     NaN     NaN   
6251     NaN  NaN   NaN   2.0    NaN    NaN    NaN    NaN     NaN     NaN   

     PELLET  FUELRATE  XGASZ  XGASA   CONFIG   RGEO    RMAG    AMIN   KAPPA  \
0      NONE       NaN    0.0    0.0       DN  1.694  1.8030  0.4101  0.9752   
1      NONE       NaN    0.0    0.0       DN  1.684  1.7970  0.4113  0.9724   
2      NONE       NaN  

In [39]:
DB5[["IP","BT","MEFF","EPS","TAUTH"]]

IP     BT  MEFF       EPS    TAUTH
0     0.2959  2.205   1.5  0.242090  0.05100
1     0.2952  2.205   1.5  0.244240  0.04902
2     0.2971  2.205   1.5  0.238807  0.06375
3     0.2959  2.205   1.5  0.242351  0.06991
4     0.2942  2.204   1.5  0.244926  0.05151
...      ...    ...   ...       ...      ...
6247 -0.2198  1.951   2.0  0.257844  0.02543
6248  0.9794  4.795   2.0  0.327343  0.17310
6249  0.9720  4.791   2.0  0.327850  0.18200
6250  1.0010 -2.506   2.0  0.297880  0.09657
6251  0.8031 -2.022   2.0  0.303578  0.11200

[6252 rows x 5 columns]

In [35]:
DB5[["IP","BT","MEFF","TAUTH"]].describe()

IP           BT         MEFF        TAUTH
count  6252.000000  6252.000000  6252.000000  6252.000000
mean     -0.400828    -1.272335     1.916213     0.177301
std       1.555441     1.845760     0.286293     0.147450
min      -5.134000    -5.821000     1.000000     0.002236
25%      -1.968693    -2.482000     1.981666     0.069788
50%       0.371800    -2.011000     2.000000     0.125950
75%       0.998000    -0.929075     2.000000     0.244868
max       2.472853     4.795000     3.890214     1.321000

In [38]:
df = DB5[["IP","BT","MEFF","EPS","TAUTH"]]
df[df.MEFF < 1.7]

IP     BT  MEFF       EPS    TAUTH
0     0.2959  2.205   1.5  0.242090  0.05100
1     0.2952  2.205   1.5  0.244240  0.04902
2     0.2971  2.205   1.5  0.238807  0.06375
3     0.2959  2.205   1.5  0.242351  0.06991
4     0.2942  2.204   1.5  0.244926  0.05151
...      ...    ...   ...       ...      ...
6161  0.3876  1.768   1.5  0.285714  0.03944
6162  0.3846  1.758   1.5  0.285714  0.03815
6163  0.3764  1.767   1.5  0.285714  0.03310
6164  0.3747  1.767   1.5  0.285714  0.03299
6165  0.3740  1.744   1.5  0.285714  0.03399

[842 rows x 5 columns]